In [1]:
import sys, os
import argparse

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "./../../../")))

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "")))

In [2]:
from fedml_api.data_preprocessing.cifar10.data_loader import load_partition_data_cifar10
from fedml_api.standalone.fedavg.my_model_trainer_classification import MyModelTrainer as MyModelTrainerCLS
from fedml_api.model.contrastive_cv.resnet_with_embedding import Resnet56
from CovaMNet import CovaMResnet56

import torch
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import random
import pickle
import wandb

# from triplet_loss import TripletLoss
from hard_triplet_loss import TripletLoss

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject


In [3]:
dataset = 'cifar10'
data_dir = "./../../../data/cifar10"
partition_method = 'hetero'
# partition_method = 'homo'
partition_alpha = 0.5
client_num_in_total = 10
batch_size = 100
total_epochs = 500

save_model_path = 'model/cs_{0}_{1}_client_{2}_new_own_mean_covaM_epochs_{3}.pt'

device = 'cuda:0'
with_cova = True

wandb.init(
            # project="federated_nas",
            project="cova",
            name="Cova_" + str(partition_method) + "_clients" + str(client_num_in_total) + "-with_new_own_mean_cova" + str(
                with_cova)
        )
# config = wandb.config
# config.learning_rate = 0.01

with open(f'dataset_{partition_method}_{client_num_in_total}.pickle', 'rb') as f:
    dataset = pickle.load(f)
# [train_data_num, test_data_num, train_data_global, test_data_global, \
# #             train_data_local_num_dict, train_data_local_dict, test_data_local_dict, \
# #             class_num]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: slimfun (use `wandb login --relogin` to force relogin)


In [4]:
# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# print(dataset[5])

In [5]:
# model = Resnet56(class_num=dataset[-1], neck='bnneck')
# model.load_state_dict(torch.load('model/cs_3_homo_client_0_oral_epochs_200.pt'))

In [6]:
# def cal_covariance(input):

#     CovaMatrix_list = []
#     mean_list = []
#     for i in range(len(input)):
#         support_set_sam = input[i]
#         support_set_sam = torch.unsqueeze(support_set_sam, 0)
#         B, C, h, w = support_set_sam.size()
        

#         support_set_sam = support_set_sam.permute(1, 0, 2, 3)
#         support_set_sam = support_set_sam.contiguous().view(C, -1)

#         mean_support = torch.mean(support_set_sam, 1, True)
#         mean_list.append(mean_support)

#         support_set_sam = support_set_sam-mean_support

#         covariance_matrix = support_set_sam@torch.transpose(support_set_sam, 0, 1)
#         covariance_matrix = torch.div(covariance_matrix, h*w*B-1)
#         CovaMatrix_list.append(covariance_matrix)

#     return CovaMatrix_list, mean_list
        
        

# cl = [torch.zeros((256,256)) for i in range(10)]
# ml = [torch.zeros((256,64)) for i in range(10)]
# lbd = 0.999
# labels = []
# def extract_features(model, data_loader, device):
#     model.to(device)
#     model.eval()
    
#     features = []
    
    
#     with torch.no_grad():
#         for batch_idx, (x, l) in enumerate(data_loader):
#             x, l = x.to(device), l.to(device)
            
#             score, feats = model(x)
#             covaM_list, mean_list = cal_covariance(feats)
#             for covaM, f, label in zip(covaM_list, feats, l):
#                 labels.append(label.cpu())
#                 for i in range(len(cl)):
#                     if label.data.cpu() == i:
#                         cl[i] = lbd * cl[i] + (1-lbd) * covaM.cpu()
#                         f = torch.unsqueeze(f, 0)
#                         B, C, h, w = f.size()

#                         f = f.permute(1, 0, 2, 3)
#                         f = f.contiguous().view(C, -1)
#                         ml[i] = lbd * ml[i] + (1-lbd) * f.cpu()

# # metrics = test(model, dataset[3], device)
# # test_correct = metrics['test_correct']/metrics['test_total']
# # test_loss = metrics['test_loss']/metrics['test_total']
# # print(metrics['test_total'])
# # print(f'test_correct: {test_correct}; test_loss: {test_loss}')
# # extract_features(model, dataset[2], device)
# # covaMs_means = [cl, ml]
# # with open(f'class_covaMs_means.pickle', 'wb') as f:
# #     pickle.dump(covaMs_means, f)

In [7]:
# with open(f'class_covaMs_means.pickle', 'rb') as f:
with open(f'new_covaM_mean.pickle', 'rb') as f:
    cms = pickle.load(f)
    
for i in range(len(cms[0])):
    cms[0][i] = cms[0][i].to(device)
#     cms[0][i] = torch.eye(256).to(device) / 256
    
# for i in range(len(cms[1])):
#     cms[1][i] = cms[1][i].to(device)
# print(cms[0][0] - cl[0])
# print(cms[1][1] - ml[1])
# from collections import Counter
# ls = np.array([int(l) for l in labels])
# counter_result = Counter(ls)
# print(Counter(ls))
# Counter({1: 1145, 2: 1138, 0: 797, 3: 661, 4: 555, 5: 539, 8: 227, 7: 38})
# print(len(cms[1]))

In [8]:
class Client(object):
    def __init__(self, client_index, train_data_local_dict, train_data_local_num_dict, test_data_local_dict, device, model):
        self.id = client_index
        self.train_data = train_data_local_dict[self.id]
        self.local_sample_number = train_data_local_num_dict[self.id]
        self.test_local = test_data_local_dict[self.id]
        
        self.device = device
        self.model = model
model1 = CovaMResnet56(class_num=dataset[-1], neck='bnneck', with_cova=with_cova)
client_1 = Client(0, dataset[5], dataset[4], dataset[6], device, model1)

In [9]:
def test(model, test_data, device):
    cl = cms[0]
    ml = cms[1]
#     support_ml = []
#     for i in range(10):
#         support_ml.append(cms[1][i].to(device))
#     ml = cms[1]
#     print(cms[0][0])
    model.to(device)
    model.eval()

    metrics = {
        'test_correct': 0,
        'test_loss': 0,
        'test_total': 0,
        'correct_cova': 0
    }

    criterion = nn.CrossEntropyLoss().to(device)

    with torch.no_grad():
        for batch_idx, (x, target) in enumerate(test_data):
            x = x.to(device)
            target = target.to(device)
            support_ml = []
            for l in target:
                support_ml.append(ml[l.data].to(device))
            pred, pred_ce_cova, feat = model(x, cl, ml=support_ml)
            loss = criterion(pred, target)
            
#             loss_ce_cova = criterion(pred_ce_cova, target)

            _, predicted = torch.max(pred, -1)
            _, predicted_ce_cova = torch.max(pred_ce_cova, -1)
            correct = predicted.eq(target).sum()
            correct_ce_cova = predicted_ce_cova.eq(target).sum()

            metrics['test_correct'] += correct.item()
            metrics['test_loss'] += loss.item() * target.size(0)
            metrics['test_total'] += target.size(0)
            metrics['correct_cova'] += correct_ce_cova.item()
            
    return metrics

def train_model(client, epochs):
#     learning_rate = 0.001
#     wd = 0.0001
#     learning_rate = 0.00035
#     wd = 0.0005

    cl = cms[0]
    ml = cms[1]
    
    margin = 0.3
    
    client.model.to(client.device)
    client.model.train()
    
    criterion = nn.CrossEntropyLoss().to(device)
    mse_loss = nn.MSELoss().to(device)
    
#     curr_lr = learning_rate
#     optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, client.model.parameters()), lr=curr_lr,
#                                          weight_decay=wd, amsgrad=True)
    optimizer = torch.optim.SGD(client.model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    epoch_loss = []
    total_step = len(client.train_data)
#     wandb.watch(client.model, log='all')
    for epoch in range(epochs):
        client.model.train()
        batch_loss = []
        for batch_idx, (x, labels) in enumerate(client.train_data):
            x, labels = x.to(device), labels.to(device)
#             print(labels.shape)
            client.model.zero_grad()
            support_ml = []
            for l in labels:
                # B, 256, 64
                support_ml.append(ml[l.data].to(device))
            cls_score, cova_score, feat = client.model(x, support_covas=cl, ml=support_ml)
#             print(score.shape)
#             print(cova_score)
            loss = criterion(cls_score, labels)
            loss = loss.to(device) + criterion(cova_score, labels).to(device)
            loss.backward()
        
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            batch_loss.append(loss.item())
            
#         if epoch % 50 == 0:
#             torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epoch))
        epoch_loss.append(sum(batch_loss) / len(batch_loss))
        scheduler.step()
        print('Client Index = {}\tEpoch: {}\tLoss: {:.6f}'.format(
            client.id, epoch, sum(batch_loss) / len(batch_loss)))
        
        metrics = test(client.model, client.test_local, client.device)
        test_correct = metrics['test_correct']/metrics['test_total']
        test_loss = metrics['test_loss']/metrics['test_total']
        correct_ce_cova = metrics['correct_cova']/metrics['test_total']
#         print(metrics['test_total'])
        print(f'test_correct: {test_correct}; test_loss: {test_loss}')
        wandb.log({
#             'epoch' : epoch,
            'train_loss': sum(batch_loss) / len(batch_loss),
            'test_correct' : test_correct,
            'test_loss' : test_loss,
            'correct_cova' : correct_ce_cova
        }, step=epoch)
        
    torch.save(client.model.state_dict(), str.format(save_model_path, client_num_in_total, partition_method, client.id, epochs))

In [10]:
train_model(client_1, 200)

Client Index = 0	Epoch: 0	Loss: 4.786476
test_correct: 0.1586; test_loss: 3.466472320556641
Client Index = 0	Epoch: 1	Loss: 3.859555
test_correct: 0.2135; test_loss: 3.184601199626923
Client Index = 0	Epoch: 2	Loss: 3.600190
test_correct: 0.2101; test_loss: 2.486977307796478
Client Index = 0	Epoch: 3	Loss: 3.407322
test_correct: 0.2278; test_loss: 2.368880548477173
Client Index = 0	Epoch: 4	Loss: 3.154656
test_correct: 0.2621; test_loss: 2.2476958739757538
Client Index = 0	Epoch: 5	Loss: 3.122354
test_correct: 0.2488; test_loss: 2.251830898523331
Client Index = 0	Epoch: 6	Loss: 2.996206
test_correct: 0.2196; test_loss: 2.4497832584381105
Client Index = 0	Epoch: 7	Loss: 2.925520
test_correct: 0.2438; test_loss: 2.474561083316803
Client Index = 0	Epoch: 8	Loss: 2.841355
test_correct: 0.2722; test_loss: 2.2827549982070923
Client Index = 0	Epoch: 9	Loss: 2.787936
test_correct: 0.2526; test_loss: 2.2142521154880526
Client Index = 0	Epoch: 10	Loss: 2.790414
test_correct: 0.2686; test_loss: 2

Client Index = 0	Epoch: 88	Loss: 1.201507
test_correct: 0.5023; test_loss: 2.1818998301029207
Client Index = 0	Epoch: 89	Loss: 1.169179
test_correct: 0.5255; test_loss: 2.1321379005908967
Client Index = 0	Epoch: 90	Loss: 1.206404
test_correct: 0.5248; test_loss: 2.1458536636829377
Client Index = 0	Epoch: 91	Loss: 1.203359
test_correct: 0.5115; test_loss: 2.2069888985157013
Client Index = 0	Epoch: 92	Loss: 1.144485
test_correct: 0.497; test_loss: 2.4136093413829802
Client Index = 0	Epoch: 93	Loss: 1.170703
test_correct: 0.5014; test_loss: 2.432875248193741
Client Index = 0	Epoch: 94	Loss: 1.176800
test_correct: 0.5273; test_loss: 2.1364772546291353
Client Index = 0	Epoch: 95	Loss: 1.137306
test_correct: 0.4821; test_loss: 2.3501340627670286
Client Index = 0	Epoch: 96	Loss: 1.044269
test_correct: 0.5298; test_loss: 2.245266420841217
Client Index = 0	Epoch: 97	Loss: 1.142396
test_correct: 0.5191; test_loss: 2.2881109929084777
Client Index = 0	Epoch: 98	Loss: 1.164139
test_correct: 0.5364;

Client Index = 0	Epoch: 175	Loss: 0.246662
test_correct: 0.5917; test_loss: 2.6711158573627474
Client Index = 0	Epoch: 176	Loss: 0.224142
test_correct: 0.5939; test_loss: 2.6914417803287507
Client Index = 0	Epoch: 177	Loss: 0.235619
test_correct: 0.5969; test_loss: 2.683759335279465
Client Index = 0	Epoch: 178	Loss: 0.240055
test_correct: 0.5982; test_loss: 2.643469064235687
Client Index = 0	Epoch: 179	Loss: 0.220918
test_correct: 0.5879; test_loss: 2.743509718179703
Client Index = 0	Epoch: 180	Loss: 0.240158
test_correct: 0.5998; test_loss: 2.660547289848328
Client Index = 0	Epoch: 181	Loss: 0.235696
test_correct: 0.5911; test_loss: 2.752983659505844
Client Index = 0	Epoch: 182	Loss: 0.222149
test_correct: 0.5959; test_loss: 2.656430376768112
Client Index = 0	Epoch: 183	Loss: 0.215241
test_correct: 0.5926; test_loss: 2.736703808307648
Client Index = 0	Epoch: 184	Loss: 0.228763
test_correct: 0.5958; test_loss: 2.671827301979065
Client Index = 0	Epoch: 185	Loss: 0.220408
test_correct: 0